In [16]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


## Import Moduls

In [103]:
import pandas as pd 
import json
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import RFE
import gdown
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

## Import Scraped Data

In [18]:
file_id = "1DDW71ZVrk8QzHe5lSOmuPgR6bumRmiCd"

# Construct the direct download link
download_link = f"https://drive.google.com/uc?id={file_id}"

# Define the output file name
output_file = "data_scraped.csv"

# Download the file from Google Drive
gdown.download(download_link, output_file, quiet=False)

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(output_file)

Downloading...
From: https://drive.google.com/uc?id=1DDW71ZVrk8QzHe5lSOmuPgR6bumRmiCd
To: /Users/bastienragueneau/Documents/je suis suisse/Jupyter/ML/albert_project/notebooks/data_scraped.csv
100%|██████████| 261k/261k [00:00<00:00, 2.86MB/s]


In [19]:
df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Caméras de surveillance extérieures présentes sur place\nLes caméras et alarmes sont situées à l'extérieur du logement pour prévenir de la sécurité du logement uniquement.,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,"TV avec Amazon Prime Video, Netflix, télévision par câble haut de gamme","Lit pour bébé : toujours dans le logement\nStandard (1,3 m de long sur 70 cm de large)",Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Preprocessing 

In [20]:
df.columns.tolist()

['Unnamed: 0',
 'prix',
 'size',
 'nb_of_travellers',
 'nb_of_bedrooms',
 'nb_of_beds',
 'nb_of_bathrooms',
 'Baignoire',
 'Shampooing',
 'Eau chaude',
 'Lave-linge',
 'Sèche-linge',
 'Équipements de base\nServiettes, draps, savon et papier toilette',
 'Cintres',
 'Draps',
 'Table à langer',
 'Chauffage',
 'Détecteur de fumée',
 'Wifi',
 'Cuisine\nEspace où les voyageurs peuvent cuisiner',
 'Réfrigérateur',
 'Four à micro-ondes',
 'Équipements de cuisine de base\nCasseroles et poêles, huile, sel et poivre',
 'Vaisselle et couverts\nBols, baguettes, assiettes, tasses, etc.',
 'Lave-vaisselle',
 'Cafetière',
 'Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.',
 'Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés',
 'Vue sur le jardin',
 'Sèche-cheveux',
 'Produits de nettoyage',
 'Savon pour le corps',
 'Gel douche',
 'Lave-linge (Gratuit) dans le logement',
 'Draps\nLinge de li

In [21]:
tv_columns = [col for col in df.columns if 'tv' in col.lower()]



In [22]:
tv_columns

['TV HD 43 pouces avec Amazon Prime Video',
 'TV avec Fire TV',
 'TV 34 pouces',
 'TV HD 43 pouces avec abonnement standard au câble',
 'TV 40 pouces avec Fire TV',
 'TV HD avec Chromecast, Amazon Prime Video, Apple TV',
 'TV HD 55 pouces avec Chromecast, Netflix',
 'TV HD 42 pouces avec Amazon Prime Video, Disney+, Netflix',
 'TV avec abonnement standard au câble',
 'TV HD 32 pouces avec Amazon Prime Video, Apple TV, Chromecast, Disney+, Fire TV, HBO Max, Hulu, Netflix, Roku',
 'TV avec Amazon Prime Video',
 'TV HD 55 pouces',
 'TV HD',
 'TV HD avec abonnement standard au câble',
 'TV avec lecteur DVD',
 'TV HD avec abonnement standard au câble, lecteur DVD, Netflix, Chromecast',
 'TV HD 43 pouces',
 'TV HD avec Netflix, télévision par câble haut de gamme',
 'TV HD 55 pouces avec Amazon Prime Video, Netflix, télévision par câble haut de gamme',
 'TV HD 55 pouces avec Netflix, abonnement standard au câble',
 'TV avec Apple TV, Amazon Prime Video',
 'TV HD 43 pouces avec Netflix',
 'TV 

In [23]:
df['TV'] = df[tv_columns].sum(axis=1)
df.drop(columns=tv_columns, inplace=True)

In [24]:
df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Caméras de surveillance extérieures présentes sur place\nLes caméras et alarmes sont situées à l'extérieur du logement pour prévenir de la sécurité du logement uniquement.,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,"Lit pour bébé : toujours dans le logement\nStandard (1,3 m de long sur 70 cm de large)",Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df.columns.tolist()

['Unnamed: 0',
 'prix',
 'size',
 'nb_of_travellers',
 'nb_of_bedrooms',
 'nb_of_beds',
 'nb_of_bathrooms',
 'Baignoire',
 'Shampooing',
 'Eau chaude',
 'Lave-linge',
 'Sèche-linge',
 'Équipements de base\nServiettes, draps, savon et papier toilette',
 'Cintres',
 'Draps',
 'Table à langer',
 'Chauffage',
 'Détecteur de fumée',
 'Wifi',
 'Cuisine\nEspace où les voyageurs peuvent cuisiner',
 'Réfrigérateur',
 'Four à micro-ondes',
 'Équipements de cuisine de base\nCasseroles et poêles, huile, sel et poivre',
 'Vaisselle et couverts\nBols, baguettes, assiettes, tasses, etc.',
 'Lave-vaisselle',
 'Cafetière',
 'Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.',
 'Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés',
 'Vue sur le jardin',
 'Sèche-cheveux',
 'Produits de nettoyage',
 'Savon pour le corps',
 'Gel douche',
 'Lave-linge (Gratuit) dans le logement',
 'Draps\nLinge de li

In [26]:
chauffage_columns = [col for col in df.columns if 'chauffage' in col.lower()]
df['Chauffage'] = df[chauffage_columns].sum(axis=1)
df.drop(columns=chauffage_columns, inplace=True)

df.head()


,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Caméras de surveillance extérieures présentes sur place\nLes caméras et alarmes sont situées à l'extérieur du logement pour prévenir de la sécurité du logement uniquement.,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,"Lit pour bébé : toujours dans le logement\nStandard (1,3 m de long sur 70 cm de large)",Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
micro_columns = [col for col in df.columns if 'micro-onde' in col.lower()]
df['Micro-onde'] = df[micro_columns].sum(axis=1)
df.drop(columns=micro_columns, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,"Lit pour bébé : toujours dans le logement\nStandard (1,3 m de long sur 70 cm de large)",Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
words = {}

for i in list(dico):
    q = i.split()

    for j in q :
        if j.lower() not in words:
            words[j.lower()] = 1
        else:
            words[j.lower()] += 1


{k: v for k, v in sorted(words.items(), key=lambda item: item[1])}


{'prix': 1}

In [29]:
words = {}

# Supposons que 'dico' est déjà défini quelque part comme une liste de phrases
for i in list(dico):
    q = i.split()

    for j in q:
        if j.lower() not in words:
            words[j.lower()] = 1
        else:
            words[j.lower()] += 1

# Trier les mots par fréquence en ordre décroissant et récupérer les 20 premiers
top_20_words = {k: v for k, v in sorted(words.items(), key=lambda item: item[1], reverse=True)[:20]}

# Afficher les 20 mots les plus fréquents
print(top_20_words)


{'prix': 1}


In [30]:
litbebe_columns = [col for col in df.columns if 'lit pour bébé' in col.lower()]
df['Lit pour bébé'] = df[litbebe_columns].sum(axis=1)
df.drop(columns=litbebe_columns, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Caméras de surveillance extérieures présentes sur place\nLes caméras et alarmes sont situées à l'extérieur du logement pour prévenir de la sécurité du logement uniquement.,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
litpara_columns = [col for col in df.columns if 'lit parapluie' in col.lower()]
df['Lit pour bébé'] = df[litpara_columns].sum(axis=1)
df.drop(columns=litpara_columns, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde,Lit pour bébé
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,1,1,0
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
{k: v for k, v in sorted(words.items(), key=lambda item: item[1], reverse=True)[:40]}

{'prix': 1}

In [33]:
[col for col in df.columns if 'cafetière' in col.lower()]

['Cafetière',
 'Cafetière : machine à expresso',
 'Cafetière : Nespresso',
 'Cafetière : cafetière manuelle',
 'Cafetière : cafetière filtre, Nespresso',
 'Cafetière : cafetière filtre',
 'Cafetière : cafetière filtre, machine à expresso, Nespresso',
 'Cafetière : cafetière filtre, cafetière à piston, Nespresso, cafetière manuelle',
 'Cafetière : Nespresso, cafetière manuelle',
 'Cafetière : cafetière à piston',
 'Cafetière : machine à expresso, Nespresso',
 'Cafetière : machine à café Keurig',
 'Cafetière : cafetière filtre, machine à expresso',
 'Cafetière : cafetière filtre, cafetière manuelle',
 'Cafetière : cafetière à piston, Nespresso']

In [34]:
x = [col for col in df.columns if 'cafetière' in col.lower()]
df['Cafetière'] = df[x].sum(axis=1)
df.drop(columns=x, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde,Lit pour bébé
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,1,1,0
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
[col for col in df.columns if 'wifi' in col.lower()]

['Wifi',
 'Vitesse du réseau wifi : 7 Mbps\nVérifié par un test de débit. Il est possible de regarder des vidéos HD en streaming.',
 'Wifi rapide (579 Mbit/s)\nVérifié par un test de débit. Il est possible de regarder des vidéos 4K en streaming et de participer à des appels vidéo sur plusieurs appareils.',
 'Wifi rapide (390 Mbit/s)\nVérifié par un test de débit. Il est possible de regarder des vidéos 4K en streaming et de participer à des appels vidéo sur plusieurs appareils.',
 'Wifi rapide (500 Mbit/s)\nVérifié par un test de débit. Il est possible de regarder des vidéos 4K en streaming et de participer à des appels vidéo sur plusieurs appareils.',
 'Wifi rapide (410 Mbit/s)\nVérifié par un test de débit. Il est possible de regarder des vidéos 4K en streaming et de participer à des appels vidéo sur plusieurs appareils.',
 'Wifi rapide (461 Mbit/s)\nVérifié par un test de débit. Il est possible de regarder des vidéos 4K en streaming et de participer à des appels vidéo sur plusieurs a

In [36]:
x = [col for col in df.columns if 'wifi' in col.lower()]
df['Wifi'] = df[x].sum(axis=1)
df.drop(columns=x, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Espace de travail dédié\nDans un espace partagé avec support pour ordinateur portable et imprimante,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde,Lit pour bébé
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,1,1,0
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [37]:
x = [col for col in df.columns if 'réfrigérateur' in col.lower()]
df['réfrigérateur'] = df[x].sum(axis=1)
df.drop(columns=x, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Cuisinière à gaz Electrolux en acier inoxydable,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde,Lit pour bébé,réfrigérateur
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,1,0,1
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,0,1,1,0,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,1
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [38]:
x = [col for col in df.columns if 'four' in col.lower()]
df['four'] = df[x].sum(axis=1)
df.drop(columns=x, inplace=True)

df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Baignoire,Shampooing,Eau chaude,...,Shampoing Black,Savon pour le corps Black,Chaise haute pliable ou transformable : toujours dans le logement\nAvec sangles ou harnais,Système audio Yamaha- Sono,Cuisinière électrique AEG,TV,Micro-onde,Lit pour bébé,réfrigérateur,four
0,0,100,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,1,0,1,0
1,1,109,60,0,0,0,0,1,1,1,...,0,0,0,0,0,1,1,0,1,1
2,2,95,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,132,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,1,1
4,4,112,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1


In [40]:
print(df.columns[:278])

Index(['Unnamed: 0', 'prix', 'size', 'nb_of_travellers', 'nb_of_bedrooms',
       'nb_of_beds', 'nb_of_bathrooms', 'Baignoire', 'Shampooing',
       'Eau chaude',
       ...
       'Chaise haute de type siège de table : disponible sur demande\nAvec sangles ou harnais et plateau',
       'Espace de travail dédié\nDans une partie du logement privée avec support pour ordinateur portable',
       'Vue sur le vignoble',
       'Système audio Bluetooth Sonos Beam + Ikea Symfonisk + UE Boom',
       'Table à langer : toujours dans le logement',
       'Caméras de surveillance extérieures présentes sur place\nUne caméra qui filme exclusivement l'entrée de l'appartement. Vous pouvez voir ce qu'elle filme dans les photos de l'annonce. Utile en cas d'effraction ou de soirées improvisées. Son usage ne sert qu'à éviter les dérives.',
       'Détecteur de fumée\n“Un détecteur dans la pièce de vie.”',
       'Détecteur de monoxyde de carbone\n“Le détecteur de fumée fait aussi office de détecteur de m

In [41]:
pd.set_option('display.max_columns', None)  # Aucune limite sur le nombre de colonnes à afficher
pd.set_option('display.max_seq_items', None)  # Aucune limite sur le nombre d'éléments dans les listes/sequences

# Afficher les noms des colonnes
print(df.columns.tolist())

['Unnamed: 0', 'prix', 'size', 'nb_of_travellers', 'nb_of_bedrooms', 'nb_of_beds', 'nb_of_bathrooms', 'Baignoire', 'Shampooing', 'Eau chaude', 'Lave-linge', 'Sèche-linge', 'Équipements de base\nServiettes, draps, savon et papier toilette', 'Cintres', 'Draps', 'Table à langer', 'Détecteur de fumée', 'Cuisine\nEspace où les voyageurs peuvent cuisiner', 'Équipements de cuisine de base\nCasseroles et poêles, huile, sel et poivre', 'Vaisselle et couverts\nBols, baguettes, assiettes, tasses, etc.', 'Lave-vaisselle', 'Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.', 'Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés', 'Vue sur le jardin', 'Sèche-cheveux', 'Produits de nettoyage', 'Savon pour le corps', 'Gel douche', 'Lave-linge (Gratuit) dans le logement', 'Draps\nLinge de lit en coton', 'Fer à repasser', 'Étendoir à linge', 'Espace de rangement pour les vêtements : placard', 'Con

In [42]:
def merge_columns(df, keyword, new_column_name):
    # Identifier les colonnes contenant le mot-clé
    columns = [col for col in df.columns if keyword in col.lower()]
    # Fusionner les informations de ces colonnes
    df[new_column_name] = df[columns].sum(axis=1)
    # Supprimer les colonnes originales
    df.drop(columns=columns, inplace=True)
    return df


# 2. Appareils de Cuisine
keywords_cuisine = ['cuisinière', 'four', 'micro-onde', 'blender', 'grille-pain', 'bouilloire', 'cuiseur', 'café', 'lave-vaisselle']
for keyword in keywords_cuisine:
    df = merge_columns(df, keyword, 'Appareils de Cuisine')

# 3. Lavage et Séchage
keywords_lavage = ['lave-linge', 'sèche-linge', 'étendoir']
for keyword in keywords_lavage:
    df = merge_columns(df, keyword, 'Lavage et Séchage')

# 4. Toilette et Hygiène
keywords_bain = ['shampooing', 'après-shampooing', 'savon', 'gel douche', 'sèche-cheveux', 'baignoire', 'douche']
for keyword in keywords_bain:
    df = merge_columns(df, keyword, 'Toilette et Hygiène')

# 5. Jeux et Livres pour Enfants
df = merge_columns(df, 'enfant', 'Jeux et Livres pour Enfants')

# 6. Literie et Confort
keywords_confort = ['draps', 'oreiller', 'climatiseur', 'ventilateur', 'chauffage']
for keyword in keywords_confort:
    df = merge_columns(df, keyword, 'Literie et Confort')

# 7. Audio et Vidéo
keywords_entertainment = ['télévision', 'audio', 'console']
for keyword in keywords_entertainment:
    df = merge_columns(df, keyword, 'Divertissement et Loisirs')

# 8. Sécurité du Logement
keywords_securite = ['détecteur', 'extincteur', 'kit', 'caméra', 'serrure']
for keyword in keywords_securite:
    df = merge_columns(df, keyword, 'Sécurité et Surveillance')

# 9. Parking et Accès
keywords_parking = ['stationnement', 'garage', 'allée', 'entrée']
for keyword in keywords_parking:
    df = merge_columns(df, keyword, 'Parking et Accès')


In [43]:
df.head()

,Unnamed: 0,prix,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Eau chaude,Cintres,Table à langer,Cuisine\nEspace où les voyageurs peuvent cuisiner,"Équipements de cuisine de base\nCasseroles et poêles, huile, sel et poivre","Vaisselle et couverts\nBols, baguettes, assiettes, tasses, etc.",Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.,Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés,Vue sur le jardin,Produits de nettoyage,Fer à repasser,Espace de rangement pour les vêtements : placard,Connexion Ethernet,Protections sur les fenêtres,Espace de travail dédié\nDans un espace commun,Wi-Fi portable,Congélateur,Verres à vin,Plaques de cuisson,Table à manger,Laverie automatique à proximité,"Patio ou balcon, accès privé",Arrière-cour partagée – Clôture partielle\nUn espace ouvert du logement généralement recouvert d'herbe,Mobilier d'extérieur,Espace repas en plein air,Logement de plain-pied\nPas d'escaliers dans le logement,Arrivée autonome,Espace de travail dédié,Boîte à clé sécurisée,Espace de rangement pour les vêtements : dressing et placard,Espace de travail dédié\nDans une pièce avec une porte,Dépôt de bagages autorisé\nPour le confort des voyageurs en cas d'arrivée anticipée ou de départ tardif,Cinéma,Clés remises par l'hôte,Stores,"Espace de rangement pour les vêtements : dressing, placard et commode",Livres et de quoi lire,Chaise haute,Jeux de société,"Patio ou balcon, accès partagé",Arrière-cour\nUn espace ouvert du logement généralement recouvert d'herbe,Chaises longues,Parking gratuit sur place,Parking gratuit dans la rue,Espace de rangement pour les vêtements,Animaux acceptés\nLes animaux d'assistance sont toujours autorisés,Logement fumeur,Chaise haute : disponible sur demande,Espace de rangement pour les vêtements : armoire,Patio ou balcon,Piano,Tourne disque,Arrière-cour privée – Clôture intégrale\nUn espace ouvert du logement généralement recouvert d'herbe,Petit déjeuner\nLe petit déjeuner est inclus,Vue panoramique sur la ville,Espace de rangement pour les vêtements : dressing,"Espace de rangement pour les vêtements : placard, armoire et commode",Ménage disponible pendant le séjour,Shampoing Rituals,Recommandations de baby-sitters,Climatisation : système split sans évacuation,Service de ménage disponible tous les jours : disponible moyennant un supplément,Shampoing Nuxe,Parking payant sur place,Shampoing RITUALS,"Espace de rangement pour les vêtements : dressing, placard, armoire et commode","Espace de travail dédié\nInclut bureau, chaise de bureau et table",Vue sur le canal,Accès au lac\nLes voyageurs peuvent se rendre à un lac en empruntant un sentier ou un quai,Après-shampoing,Espace de rangement pour les vêtements : placard et armoire,Machine à pain,Climatisation,Vue sur la cour,Arrière-cour privée\nUn espace ouvert du logement généralement recouvert d'herbe,Vélos,Espace de rangement pour les vêtements : placard et commode,Chaise haute pliable ou transformable : toujours dans le logement,Cheminée,Shampoing Nivea,Station de recharge pour véhicules électriques\nLes voyageurs peuvent recharger leur véhicule électrique sur place,Chaise haute de type rehausseur,Espace de travail dédié\nInclut bureau et table,Caches-prises,Barbecue : électrique,"Ustensiles de barbecue\nBarbecue, charbon, brochettes en bambou ou en métal, etc.",Barbecue,Espace de rangement pour les vêtements : commode,Hamac,Chaise haute : toujours dans le logement\nAvec sangles ou harnais,Bidet,Appareils de fitness : tapis de yoga,Protections sur les coins de tables,Arrière-cour partagée – Clôture intégrale\nUn espace ouvert du logement généralement recouvert d'herbe,Salle de sport partagée à proximité,"Espace de travail dédié\nDans un espace partagé avec chaise ergonomique, support pour ordinateur portable et imprimante",Espace de rangement pour les vêtements : armoire et commode,Barbecue : au charbon de bois,Moustiquaire,

In [44]:
len(df.columns)

161

## Implementation Modeles 

In [65]:
X = df.drop(columns='prix')
X = X.drop(columns='Unnamed: 0')
y = df['prix']

In [66]:
X

,size,nb_of_travellers,nb_of_bedrooms,nb_of_beds,nb_of_bathrooms,Eau chaude,Cintres,Table à langer,Cuisine\nEspace où les voyageurs peuvent cuisiner,"Équipements de cuisine de base\nCasseroles et poêles, huile, sel et poivre","Vaisselle et couverts\nBols, baguettes, assiettes, tasses, etc.",Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.,Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés,Vue sur le jardin,Produits de nettoyage,Fer à repasser,Espace de rangement pour les vêtements : placard,Connexion Ethernet,Protections sur les fenêtres,Espace de travail dédié\nDans un espace commun,Wi-Fi portable,Congélateur,Verres à vin,Plaques de cuisson,Table à manger,Laverie automatique à proximité,"Patio ou balcon, accès privé",Arrière-cour partagée – Clôture partielle\nUn espace ouvert du logement généralement recouvert d'herbe,Mobilier d'extérieur,Espace repas en plein air,Logement de plain-pied\nPas d'escaliers dans le logement,Arrivée autonome,Espace de travail dédié,Boîte à clé sécurisée,Espace de rangement pour les vêtements : dressing et placard,Espace de travail dédié\nDans une pièce avec une porte,Dépôt de bagages autorisé\nPour le confort des voyageurs en cas d'arrivée anticipée ou de départ tardif,Cinéma,Clés remises par l'hôte,Stores,"Espace de rangement pour les vêtements : dressing, placard et commode",Livres et de quoi lire,Chaise haute,Jeux de société,"Patio ou balcon, accès partagé",Arrière-cour\nUn espace ouvert du logement généralement recouvert d'herbe,Chaises longues,Parking gratuit sur place,Parking gratuit dans la rue,Espace de rangement pour les vêtements,Animaux acceptés\nLes animaux d'assistance sont toujours autorisés,Logement fumeur,Chaise haute : disponible sur demande,Espace de rangement pour les vêtements : armoire,Patio ou balcon,Piano,Tourne disque,Arrière-cour privée – Clôture intégrale\nUn espace ouvert du logement généralement recouvert d'herbe,Petit déjeuner\nLe petit déjeuner est inclus,Vue panoramique sur la ville,Espace de rangement pour les vêtements : dressing,"Espace de rangement pour les vêtements : placard, armoire et commode",Ménage disponible pendant le séjour,Shampoing Rituals,Recommandations de baby-sitters,Climatisation : système split sans évacuation,Service de ménage disponible tous les jours : disponible moyennant un supplément,Shampoing Nuxe,Parking payant sur place,Shampoing RITUALS,"Espace de rangement pour les vêtements : dressing, placard, armoire et commode","Espace de travail dédié\nInclut bureau, chaise de bureau et table",Vue sur le canal,Accès au lac\nLes voyageurs peuvent se rendre à un lac en empruntant un sentier ou un quai,Après-shampoing,Espace de rangement pour les vêtements : placard et armoire,Machine à pain,Climatisation,Vue sur la cour,Arrière-cour privée\nUn espace ouvert du logement généralement recouvert d'herbe,Vélos,Espace de rangement pour les vêtements : placard et commode,Chaise haute pliable ou transformable : toujours dans le logement,Cheminée,Shampoing Nivea,Station de recharge pour véhicules électriques\nLes voyageurs peuvent recharger leur véhicule électrique sur place,Chaise haute de type rehausseur,Espace de travail dédié\nInclut bureau et table,Caches-prises,Barbecue : électrique,"Ustensiles de barbecue\nBarbecue, charbon, brochettes en bambou ou en métal, etc.",Barbecue,Espace de rangement pour les vêtements : commode,Hamac,Chaise haute : toujours dans le logement\nAvec sangles ou harnais,Bidet,Appareils de fitness : tapis de yoga,Protections sur les coins de tables,Arrière-cour partagée – Clôture intégrale\nUn espace ouvert du logement généralement recouvert d'herbe,Salle de sport partagée à proximité,"Espace de travail dédié\nDans un espace partagé avec chaise ergonomique, support pour ordinateur portable et imprimante",Espace de rangement pour les vêtements : armoire et commode,Barbecue : au charbon de bois,Moustiquaire,Compacteur de dé

In [67]:
y

0      100
1      109
2       95
3      132
4      112
      ... 
263    150
264    109
265    120
266    210
267    101
Name: prix, Length: 268, dtype: int64

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
#accuracy = accuracy_score(y_true, y_pred)

#print(f'Accuracy: {accuracy}')

In [85]:
model = RandomForestClassifier().fit(X_train, y_train)
y_pred = model.predict(X_test)

In [86]:
print("R2:", r2_score(y_test, y_pred))

R2: -0.09609007678971815


In [87]:
from sklearn.metrics import mean_squared_error

In [88]:
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)

RMSE: 44.2428064879018


In [89]:
y_true = y_test

accuracy = accuracy_score(y_true, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy: 0.2777777777777778


### With RFE selector of features : 

In [95]:
model = RandomForestClassifier()

selector = RFE(model, n_features_to_select=18)
selector = selector.fit(X_train, y_train)

selector

RFE(estimator=RandomForestClassifier(), n_features_to_select=18)

In [96]:
selected_columns = X.columns[selector.support_].tolist()

print("Colonnes sélectionnées par RFE :", selected_columns)

Colonnes sélectionnées par RFE : ['size', 'Ascenseur\nLa maison ou le bâtiment dispose d’un ascenseur d’une profondeur minimale de 132cm et d’une porte d’une largeur minimale de 81cm.', 'Séjours longue durée autorisés\nSéjours de 28 jours ou plus autorisés', 'Produits de nettoyage', 'Fer à repasser', 'Table à manger', 'Laverie automatique à proximité', 'Arrivée autonome', 'Espace de travail dédié', "Clés remises par l'hôte", 'Stores', 'Livres et de quoi lire', 'Parking gratuit sur place', 'réfrigérateur', 'Appareils de Cuisine', 'Lavage et Séchage', 'Jeux et Livres pour Enfants', 'Parking et Accès']


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = selector.predict(X_test)
y_true = y_test

In [99]:
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)

RMSE: 54.149997435108176


## Final Pipeline :

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'Support Vector Regressor': SVR()
}

results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  
        ('regressor', model)
    ])
    
    pipeline.fit(X_train, y_train)
    
    score = pipeline.score(X_test, y_test)  # R^2 score sur les données de test
    results[name] = score
    print(f"Score du modèle {name} sur les données de test : {score:.2f}")

print("Résultats finaux:")
for model, score in results.items():
    print(f"{model}: R^2 = {score:.2f}")

Score du modèle Random Forest Regressor sur les données de test : 0.24
Score du modèle Gradient Boosting Regressor sur les données de test : 0.28
Score du modèle Support Vector Regressor sur les données de test : 0.02
Résultats finaux:
Random Forest Regressor: R^2 = 0.24
Gradient Boosting Regressor: R^2 = 0.28
Support Vector Regressor: R^2 = 0.02
